<a href="https://colab.research.google.com/github/Ehsan1997/ObjectDetection-FRCNN-YOLO-Retina/blob/master/FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
zip_path = '/content/drive/My Drive/Datasets/SIMS-dataset.zip'
!unzip '$zip_path' -d ./data/
!unzip -q 'data/SIMS dataset/images.zip' -d data
!unzip -q 'data/SIMS dataset/Annotations_in_3_formats.zip' -d data

Archive:  /content/drive/My Drive/Datasets/SIMS-dataset.zip
  inflating: ./data/SIMS dataset/test.txt  
  inflating: ./data/SIMS dataset/validation.txt  
  inflating: ./data/SIMS dataset/training.txt  
  inflating: ./data/SIMS dataset/Assignment-2_modified scope.pdf  
  inflating: ./data/SIMS dataset/Annotations_in_3_formats.zip  
  inflating: ./data/SIMS dataset/images.zip  


In [0]:
import os, csv, PIL

def create_csv(txt_path, output_path, class_mapping):
  """
  Creates a csv file that can be used with keras-retinanet csv generator
  (Dependencies: [os, csv, PIL])
  """
  # A subfunction to read annotations file
  def read_ann(ann_path):
    with open(ann_path, 'r') as f:
      lines = f.readlines()
    return lines

  # A subfunction to convert percentage centre 
  # co-ordinates to x1, y1, x2, y2 absolute co-ordinates
  def get_abs_coord(coords, im_w, im_h):
    n_coords = coords.copy()
    n_coords[0] = (coords[0] - (coords[2]/2)) * im_w
    n_coords[2] = n_coords[0] + (coords[2] * im_w)
    n_coords[1] = (coords[1] - (coords[3]/2)) * im_h
    n_coords[3] = n_coords[1] + (coords[3] * im_h)

    return list(map(int, n_coords))

  # Class ID to name mapping
  id_to_name = class_mapping

  # create_csv continues.
  # A bit of mangling with the path
  root = '/content/data'
  # CSV list to keep track of the lines needed to be written to the csv
  csv_list = []
  with open(txt_path, 'r') as f:
    line = f.readline()
    while line:
      # Get images path
      im_path = line.strip()
      abs_im_path = root + '/' + "/".join(line.strip().split('/')[1:])
      # Get path of annotations for the image
      im_txt_path = os.path.splitext(abs_im_path)[0] + '.txt'
      # Get the annotations
      anns = read_ann(im_txt_path)
      for ann in anns:
        items = ann.strip().split(' ')
        c_id, bb = int(items[0]), list(map(float, items[1:]))
        # Get the size of the image
        w, h = PIL.Image.open(abs_im_path).size
        # Get absolute co-ordinates
        bb = get_abs_coord(bb, w, h)
        # Row for csv file, converting x,y,w,h to x,y,x1, y1
        csv_list.append([abs_im_path, *bb, id_to_name[c_id]])
      # Read next line
      line = f.readline()

    with open(output_path, 'w', newline="") as f:
      writer = csv.writer(f)
      writer.writerows(csv_list)
        

In [0]:
class_mapping = ['Car', 'Truck', 'Van', 'LongVehicle', 'Bus', 
                'Airliner', 'Propeller Aircraft', 'Trainer Aircraft', 'Chartered Aircraft',
                'Fighter Aircraft', 'Others', 'Stair Truck', 'Pushback Truck',
                'Helicopter', 'Boat']
class_mapping = dict(enumerate(class_mapping))

create_csv('data/SIMS dataset/training.txt', 'data/training.csv', class_mapping)
create_csv('data/SIMS dataset/test.txt', 'data/test.csv', class_mapping)
create_csv('data/SIMS dataset/validation.txt', 'data/validation.csv', class_mapping)

class_csv = list(map(lambda x: (x[1], x[0]), class_mapping.items()))

with open('data/classes.csv', 'w', newline="") as f:
  writer = csv.writer(f)
  writer.writerows(class_csv)

In [5]:
!git clone https://github.com/kbardool/keras-frcnn
!mv keras-frcnn/* .
!rm -r keras-frcnn
!pip install -r requirements.txt

Cloning into 'keras-frcnn'...
remote: Enumerating objects: 589, done.
remote: Total 589 (delta 0), reused 0 (delta 0), pack-reused 589
Receiving objects: 100% (589/589), 173.38 KiB | 771.00 KiB/s, done.
Resolving deltas: 100% (402/402), done.
     |████████████████████████████████| 204kB 8.3MB/s 
  Created wheel for Keras: filename=Keras-2.0.3-cp36-none-any.whl size=232962 sha256=afb9ddd7a2b56c45a93df35eba5108f2fd8ddca96e71b4973f5f3455c6c817af
  Stored in directory: /root/.cache/pip/wheels/a6/fb/de/faea9e49d563a35f198c6dede7f9260074b5beb8f9bffaaaa1
Successfully built Keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.3 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [0]:
!python train_frcnn.py -o simple -p data/training.csv --num_epochs 20

Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'Airliner': 756,
 'Boat': 3965,
 'Bus': 1337,
 'Car': 8861,
 'Chartered Aircraft': 485,
 'Fighter Aircraft': 41,
 'Helicopter': 44,
 'LongVehicle': 1241,
 'Others': 595,
 'Propeller Aircraft': 154,
 'Pushback Truck': 141,
 'Stair Truck': 317,
 'Trainer Aircraft': 490,
 'Truck': 1936,
 'Van': 3510,
 'bg': 0}
Num classes (including bg) = 16
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 2694
Num val samples 539







Instructions for updating:
keep_dims is deprecated, use keepdims instead
loading weights from resnet50_weights_tf_dim_ordering_tf_kernels.h5
Could not load pretrained model weights. Weights can be found in the keras application folder 		https://github.com/fchollet/keras/tree/master/keras/applications


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
keep_d

In [19]:
!python test_frcnn.py -p images/0009.jpg

Using TensorFlow backend.
{0: 'Car', 1: 'Chartered Aircraft', 2: 'Airliner', 3: 'Van', 4: 'Others', 5: 'Stair Truck', 6: 'Pushback Truck', 7: 'Bus', 8: 'Truck', 9: 'Helicopter', 10: 'Propeller Aircraft', 11: 'Boat', 12: 'Trainer Aircraft', 13: 'LongVehicle', 14: 'Fighter Aircraft', 15: 'bg'}







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading weights from ./model_frcnn.hdf5



2020-05-14 19:50:06.543637: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-05-14 19:50:06.543876: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xbed01c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-14 19:50:06.543913: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-05-14 19:50:06.545808: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-14 

In [0]:
!python measure_map.py -o simple -p data/validation.csv

In [0]:
!rm -r test_images
!mkdir test_images
!mkdir results_imgs
!cp data/images/0300.jpg test_images
!python test_frcnn.py -p test_images